In [2]:
import cv2
import mediapipe as mp
import pyautogui
import math
import time

In [80]:
# Function to draw circles on landmarks
def draw_landmarks(image, landmarks):
    points = [landmarks[4],landmarks[8],landmarks[12]]
    for landmark in points:
        x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)

In [13]:
# MediaPipe initialization
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Go to display and change this variables
SCREEN_WIDTH, SCREEN_HEIGHT = 1920, 1080

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Configure performance optimizations
FRAME_WIDTH = 480
FRAME_HEIGHT = 350
THROTTLE_TIME = 0.001  # Delay between cursor movements in seconds

# Initialize variables for cursor movement
last_move_time = time.time()
mouse_button_pressed = False

with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.9) as hands:
    while cap.isOpened():
        success, frame = cap.read()
        #frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
        image = cv2.flip(frame, 1)
        if not success:
            break

        # Convert the BGR image to RGB and process it
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        # Detect hand and perform actions
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = hand_landmarks.landmark
                
                # If you want to draw landmarks
                # draw_landmarks(image, landmarks)

                # Handling right click
                click = landmarks[4].y - landmarks[8].y
                if click < 0.03:
                    pyautogui.leftClick()

                # Handling mouse press
                press = landmarks[12].y - landmarks[8].y
                if press < 0.03:
                    if not mouse_button_pressed:
                        pyautogui.mouseDown()
                        mouse_button_pressed = True
                else:
                    if mouse_button_pressed:
                        pyautogui.mouseUp()
                        mouse_button_pressed = False

                # For moving cursor
                if time.time() - last_move_time >= THROTTLE_TIME:
                    x = int(landmarks[8].x * SCREEN_WIDTH)
                    y = int(landmarks[8].y * SCREEN_HEIGHT)
                    pyautogui.moveTo(x, y)
                    last_move_time = time.time()

        cv2.imshow('Hand Detection', image)

        if cv2.waitKey(5) & 0xFF == 27:  # ESC key to exit
            break

cap.release()
cv2.destroyAllWindows()

In [5]:
cap.release()
cv2.destroyAllWindows()